In [3]:
import pandas as pd

Data is taken from the Kaggle competition - check https://www.kaggle.com/competitions/m5-forecasting-accuracy/data
for explanations / descriptions.

In [8]:
# general settings
class CFG:
    data_folder = './data/'


In [9]:
# calendar data
calendar_df = pd.read_csv(CFG.data_folder + 'calendar.csv', parse_dates=['date'])
calendar_df = calendar_df.loc[:, ['date', 'wm_yr_wk', 'd']]
calendar_df.head(3)

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3


In [17]:
# sales data
df = pd.read_csv(CFG.data_folder + 'sales_train_evaluation.csv')
# subset to single item for demo purposes
df = df.loc[df.item_id=='FOODS_3_819']
df_T = df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
df_T.drop(columns=['id'], inplace=True)

# merge sales with calendar to get dates
sales_df = df_T.merge(calendar_df, left_on='variable', right_on='d', how='left')
sales_df.rename(columns={'value': 'sales_qty'}, inplace=True)
df = sales_df.loc[sales_df.date >= '2014-01-01', ['date', 'store_id', 'sales_qty']]
# add a state column
df['state_id'] = df['store_id'].str[:2]
# rearrange columns
df = df[['date', 'state_id', 'store_id', 'sales_qty']]
# rename columns - unfortunately hierarchicalforecasting package requires specific names (hardcoded)
# my educated guess is that it's a legacy from Prophet: https://facebook.github.io/prophet/
df.rename(columns={'date': 'ds', 'sales_qty': 'y'}, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,ds,state_id,store_id,y
0,2014-01-01,CA,CA_1,3
1,2014-01-01,CA,CA_2,0
2,2014-01-01,CA,CA_3,7
3,2014-01-01,CA,CA_4,0
4,2014-01-01,TX,TX_1,0
...,...,...,...,...
8725,2016-05-22,TX,TX_2,0
8726,2016-05-22,TX,TX_3,1
8727,2016-05-22,WI,WI_1,7
8728,2016-05-22,WI,WI_2,0


In [18]:
# store the prepared data
df.to_csv(CFG.data_folder + 'm04_sales_data_prepared.csv', index=False)